# Projet DALAS 
## CLleanup before MLN
### SOYKOK Aylin - CELIK Simay


We scraped the data separately so we have to fix their forms etc.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import numpy as np

In [2]:
df_fast = pd.read_pickle('../data/fastfashion_clean.pkl')
df_slow = pd.read_pickle('../data/slowfashion_clean.pkl')
# slow_label = df_slow['Brand Type'].tolist()
# df_slow = df_slow.drop(columns=['Brand Type'])
fast_label = ['Fast Fashion']*len(df_fast)
df_fast['Brand Type'] = fast_label
df_fast = df_fast.rename(columns={'Pays de fabrication':'Country','Material':'Materials'})

In [3]:
print(df_slow.columns)
df_slow.head()

Index(['Product Name', 'Price', 'Materials', 'Factory', 'Factory Link',
       'Product Type', 'Country', 'Brand Type'],
      dtype='object')


,Product Name,Price,Materials,Factory,Factory Link,Product Type,Country,Brand Type
0,The Original Curvy Cheeky® Jean,118.0,"[Organic cotton, Elastane]",Made at our Hirdaramani Factory in Sri Lanka.,Factory Link Not Found,Jeans,[Sri Lanka],Slow Fashion
1,The Way-High® Jean,142.0,"[Organic cotton, Elastane]","Made in Bien Hoa, VN",/factories/denim-saitex,Jeans,[Vietnam],Slow Fashion
2,The A-Line Denim Short,81.0,[Organic cotton],Made at our Hirdaramani Factory in Sri Lanka.,Factory Link Not Found,Jeans,[Sri Lanka],Slow Fashion
3,The Summer Barrel Jean,142.0,"[Organic cotton, Tencel lyocell with refibra]",Made at Phong Phu International Joint Stock Co...,Factory Link Not Found,Jeans,[Vietnam],Slow Fashion
4,The Cheeky Relaxed Straight Jean,142.0,"[Organic cotton, Elastane]","Made in Bien Hoa, VN",/factories/denim-saitex,Jeans,[Vietnam],Slow Fashion


In [4]:
df_fast.columns
df_fast.head()

,Product Type,Product Name,Price,Materials,Link,Informations,Pays concernés,Impression,Country,Teinture,Tissage,Fabrication,Pays d’origine du matériau,Brand Type
0,Jeans,Pantalon cargo en denim,29.99,"[ Coton, Coton recyclé]",https://www2.hm.com/fr_fr/productpage.11989210...,[Pays de fabrication\nBangladesh],[Bangladesh],[],[Bangladesh],[],[],[],[],Fast Fashion
1,Jeans,Ultra High Ankle Jeggings,29.99,"[ Coton, Coton recyclé, Polyester, Polyeste...",https://www2.hm.com/fr_fr/productpage.11524570...,"[Pays d’origine du matériau\nChine, Pays de fa...","[Chine, Chine]",[],[Chine],[],[],[],[Chine],Fast Fashion
2,Jeans,Wide High Jeans,29.99,"[ Coton, Coton recyclé, Polyester, Polyeste...",https://www2.hm.com/fr_fr/productpage.08718890...,[Pays de fabrication\nBangladesh],[Bangladesh],[],[Bangladesh],[],[],[],[],Fast Fashion
3,Jeans,Flared Low Jeans,25.99,"[ Coton, Coton recyclé, Polyester, Élasthanne]",https://www2.hm.com/fr_fr/productpage.11834980...,"[Teinture\nPakistan, Tissage\nPakistan, Fabric...","[Pakistan, Pakistan, Pakistan]",[],[],[Pakistan],[Pakistan],[Pakistan],[],Fast Fashion
5,Jeans,Wide High Cropped Jeans,25.99,"[ Coton, Coton recyclé, Polyester, Polyeste...",https://www2.hm.com/fr_fr/productpage.11991910...,[Pays de fabrication\nBangladesh],[Bangladesh],[],[Bangladesh],[],[],[],[],Fast Fashion


In [5]:
df_fast = df_fast[df_fast['Country'].apply(lambda x: x != [])]
df_slow = df_slow[df_slow['Country'].apply(lambda x: x != [])]

df_fast['Country'].explode().value_counts()

Country
Chine         649
Bangladesh    567
Turquie       230
Birmanie      197
Cambodge      128
Inde           87
Indonésie      68
Vietnam        55
Pakistan       26
Maroc          18
Roumanie       13
Bulgaria        8
Mexique         7
Guatemala       1
Kenya           1
Ethiopie        1
Name: count, dtype: int64

In [6]:
#!pip install googletrans==4.0.0-rc1

In [7]:
french_to_english = {
    'Bangladesh': 'Bangladesh',
    'Chine': 'China',
    'Turquie': 'Turkey',
    'Birmanie': 'Myanmar',
    'Inde': 'India',
    'Cambodge': 'Cambodia',
    'Indonésie': 'Indonesia',
    'Pakistan': 'Pakistan',
    'Vietnam': 'Vietnam',
    'Maroc': 'Morocco',
    'Bulgaria': 'Bulgaria',
    'Roumanie': 'Romania',
    'Mexique': 'Mexico',
    'Guatemala': 'Guatemala',
    'Italy': 'Italy',
    'Honduras': 'Honduras',
    'Ethiopie': 'Ethiopia',
    'Kenya': 'Kenya'
}
df_fast['Country'] = df_fast['Country'].apply(lambda lst: [french_to_english.get(x, x) for x in lst])
print(df_fast['Country'])


0      [Bangladesh]
1           [China]
2      [Bangladesh]
5      [Bangladesh]
6      [Bangladesh]
           ...     
171         [China]
172    [Bangladesh]
173         [China]
174    [Bangladesh]
175         [China]
Name: Country, Length: 1789, dtype: object


In [8]:
!pip install deep-translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.5 MB/s eta 0:00:00


In [9]:
from deep_translator import GoogleTranslator
df_fast = df_fast[df_fast['Materials'].apply(lambda x: x != [])]
exploded_materials = df_fast['Materials'].explode()

# Translate each material to English, skipping None or NaN values
translator = dict()
for i, material in enumerate(exploded_materials.unique()):
    if material is None or pd.isna(material):
        continue
    translation = GoogleTranslator(source='french', target='english').translate(material)
    print(f'{material} -> {translation}')
    translator[material] = translation


 Coton -> Cotton
 Coton recyclé -> Recycled cotton
 Polyester -> Polyester
 Polyester recyclé -> Recycled polyester
 Viscose -> Viscose
 Élasthanne -> Elastane
 Copolyester -> Copolyester
 Polyuréthane -> Polyurethane
 Élastomultiester -> Elastomultiester
 Circulose® -> Circulose®
 Lyocell -> Lyocell
 Lyocell Tencel™ -> Lyocell Tencel™
 Coton biologique -> Organic cotton
 Polyamide -> Polyamide
 Modal -> Modal
 Viscose LivaEco™ -> LivaEco™ Viscose
 Viscose LENZING™ ECOVERO™ -> LENZING™ ECOVERO™ Viscose
 Polyamide recyclé -> Recycled polyamide
 Soie -> Silk
 Lin -> Linen
 Acrylique -> Acrylic
 Laine -> Wool
 Mohair -> Mohair
 Fibre métallisée -> Metallized fiber
 Autres fibres -> Other fibers
 Laine recyclée -> Recycled wool
 Polyester recyclé d'origine textile -> Recycled polyester of textile origin
 Cachemire -> Cashmere
 Cuir -> Leather
 Polytéréphtalate d'éthylène -> Polyethylene terephthalate
 Laine de yack -> Yak wool
 Acétate -> Acetate
 Polystyrène -> Polystyrene


In [10]:
df_fast['Materials'] = df_fast['Materials'].apply(lambda lst: [translator.get(x, x) for x in lst])
print(df_fast['Materials'].explode().value_counts())

Materials
Polyester                               956
Cotton                                  917
Elastane                                806
Recycled polyester                      733
Recycled cotton                         438
Viscose                                 401
Polyamide                               283
LivaEco™ Viscose                        165
Linen                                   118
Recycled polyamide                       95
Recycled polyester of textile origin     67
LENZING™ ECOVERO™ Viscose                48
Wool                                     40
Organic cotton                           31
Acrylic                                  30
Silk                                     27
Lyocell                                  23
Lyocell Tencel™                          18
Modal                                    14
Polyurethane                             12
Metallized fiber                         11
Leather                                   5
Elastomultiester      

In [13]:
# unifying material names with sustainable brand
df_fast['Materials'] = df_fast['Materials'].apply(lambda lst: ["Lenzing ecovero viscose" if x == "LENZING™ ECOVERO™ Viscose" else x for x in lst])
df_fast['Materials'] = df_fast['Materials'].apply(lambda lst: ["Tencel lyocell" if x == "Lyocell Tencel™" else x for x in lst])

print(df_fast['Materials'].explode().value_counts())

Materials
Polyester                               956
Cotton                                  917
Elastane                                806
Recycled polyester                      733
Recycled cotton                         438
Viscose                                 401
Polyamide                               283
LivaEco™ Viscose                        165
Linen                                   118
Recycled polyamide                       95
Recycled polyester of textile origin     67
Lenzing ecovero viscose                  48
Wool                                     40
Organic cotton                           31
Acrylic                                  30
Silk                                     27
Lyocell                                  23
Tencel lyocell                           18
Modal                                    14
Polyurethane                             12
Metallized fiber                         11
Leather                                   5
Elastomultiester      

In [14]:
minitrans = {'Hauts':'Tops'}
df_fast['Product Type'] = df_fast['Product Type'].apply(lambda x: minitrans.get(x,x))
print(df_fast['Product Type'].explode().value_counts())
print(df_slow['Product Type'].explode().value_counts())

Product Type
Tops          944
Pantalons     504
Jeans         202
Sweatshirt    139
Name: count, dtype: int64
Product Type
Tops         212
Pantalons    102
Jeans         52
Name: count, dtype: int64


In [15]:
columns = ['Product Type','Product Name', 'Price', 'Materials', 'Country', 'Brand Type']
df_slow_selected = df_slow[columns]
df_fast_selected = df_fast[columns]
merged_df = pd.concat([df_fast_selected, df_slow_selected], axis=0)
merged_df.head()

,Product Type,Product Name,Price,Materials,Country,Brand Type
0,Jeans,Pantalon cargo en denim,29.99,"[Cotton, Recycled cotton]",[Bangladesh],Fast Fashion
1,Jeans,Ultra High Ankle Jeggings,29.99,"[Cotton, Recycled cotton, Polyester, Recycled ...",[China],Fast Fashion
2,Jeans,Wide High Jeans,29.99,"[Cotton, Recycled cotton, Polyester, Recycled ...",[Bangladesh],Fast Fashion
5,Jeans,Wide High Cropped Jeans,25.99,"[Cotton, Recycled cotton, Polyester, Recycled ...",[Bangladesh],Fast Fashion
6,Jeans,Curvy Fit Slim High Jeans,35.99,"[Cotton, Recycled cotton, Polyester, Elastane]",[Bangladesh],Fast Fashion


In [16]:
merged_df['Product Type'].value_counts()

Product Type
Tops          1156
Pantalons      606
Jeans          254
Sweatshirt     139
Name: count, dtype: int64

In [17]:
merged_df = merged_df[merged_df['Product Type'].apply(lambda x: x != 'Sweatshirt')]
merged_df['Product Type'].value_counts()

Product Type
Tops         1156
Pantalons     606
Jeans         254
Name: count, dtype: int64

In [18]:
merged_df.to_pickle('base.pkl')